# K26 - Heated Wall

Interface at 90°.  
Equal fluid densities => simplified setting  
Also no Heat capacity => infinitely fast heat conduction  

In [ ]:
#r "..\..\..\src\L4-application\BoSSSpad\bin\Release\net5.0\BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

## Setup Workflowmanagement, Batchprocessor and Database

In [ ]:
ExecutionQueues

index,type,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode,AllowedDatabasesPaths,BatchInstructionDir
0,BoSSS.Application.BoSSSpad.MsHPC2012Client,\\hpccluster\hpccluster-scratch\rieckmann\binaries,True,<null>,dotnet,FDY\rieckmann,DC2,<null>,Normal,True,"[ { \\hpccluster\hpccluster-scratch\rieckmann == : LocalMountPath: \\hpccluster\hpccluster-scratch\rieckmann, PathAtRemote: } ]",
1,BoSSS.Application.BoSSSpad.MsHPC2012Client,\\hpccluster\hpccluster-scratch\rieckmann\binaries,True,HPCCLUSTER,dotnet,FDY\rieckmann,DC2,[ hpccluster ],Normal,True,"[ { \\hpccluster\hpccluster-scratch\rieckmann\BoSSS_DB == : LocalMountPath: \\hpccluster\hpccluster-scratch\rieckmann\BoSSS_DB, PathAtRemote: } ]",
2,BoSSS.Application.BoSSSpad.MsHPC2012Client,\\hpccluster\hpccluster-scratch\rieckmann\binaries,True,HPCCLUSTER2,dotnet,FDY\rieckmann,DC2,[ hpccluster2 ],Normal,True,"[ { \\hpccluster\hpccluster-scratch\rieckmann\BoSSS_DB == : LocalMountPath: \\hpccluster\hpccluster-scratch\rieckmann\BoSSS_DB, PathAtRemote: } ]",
3,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,C:\Users\rieckmann\AppData\Local\BoSSS-LocalJobs,False,<null>,dotnet,,,,,,,<null>


In [ ]:
static var myBatch = BoSSSshell.GetDefaultQueue();

In [ ]:
BoSSSshell.WorkflowMgm.Init($"HeatedWall_Simple");

Project name is set to 'HeatedWall_Simple'.


In [ ]:
static var myDb = BoSSSshell.WorkflowMgm.DefaultDatabase;

In [ ]:
myDb.Path

Opening existing database '\\hpccluster\hpccluster-scratch\rieckmann\XNSFE_HeatedWall'.


\\hpccluster\hpccluster-scratch\rieckmann\XNSFE_HeatedWall

## Postprocessing

In [ ]:
var sessions = BoSSSshell.WorkflowMgm.Sessions.Where(s => s.SuccessfulTermination == true).ToList();

In [ ]:
sessions

#0: HeatedWall_Simple	HeatedWall_Simple_res:4_p:2_Q:0	10/06/2021 13:47:26	6fec0a28...
#1: HeatedWall_Simple	HeatedWall_Simple_res:4_p:2_Q:0.05	10/06/2021 13:47:55	9fef9d0e...
#2: HeatedWall_Simple	HeatedWall_Simple_res:4_p:2_Q:0.1	10/06/2021 13:48:29	d436ecf9...
#3: HeatedWall_Simple	HeatedWall_Simple_res:4_p:2_Q:0.2	10/06/2021 13:48:35	71449fd9...


In [ ]:
var dataCL = sessions.ReadLogDataForMovingContactLine()[1];
var dataEvap = sessions.ReadLogDataForXNSE("Massflux");
dataCL.Pick(4).dataGroups.ForEach(s => s.Values = s.Values.Select(v => 180 - v ).ToArray());

Plot2Ddata evapFluxes = new Plot2Ddata();
evapFluxes.dataGroups = new Plot2Ddata.XYvalues[dataEvap.Pick(3).dataGroups.Length]; // same as total evaporative heatflux * dt, as rho = hvap = 1
Plot2Ddata bndFluxes = new Plot2Ddata();
bndFluxes.dataGroups = new Plot2Ddata.XYvalues[dataCL.Pick(1).dataGroups.Length]; 
Plot2Ddata.XYvalues[] Vals = new Plot2Ddata.XYvalues[dataCL.Pick(4).dataGroups.Length];
Plot2Ddata clVeloY = new Plot2Ddata();
clVeloY.dataGroups = new Plot2Ddata.XYvalues[dataCL.Pick(3).dataGroups.Length]; 
int dashType = 0;
for(int i = 0; i<sessions.Count;i++){
    sessions[i].KeysAndQueries.TryGetValue("id:PressureRatio", out object color);
    color = color == null ? 7 : Convert.ToInt32(color);
    dataCL.ForEach(p => p.dataGroups[i].Name = "q:" + Convert.ToString(sessions[i].KeysAndQueries["id:HeatFlux"]));
    dataEvap.ForEach(p => p.dataGroups[i].Name = "q:" + Convert.ToString(sessions[i].KeysAndQueries["id:HeatFlux"]));
    PlotFormat PF = new PlotFormat(lineColor: (LineColors)color, dashType: ((DashTypes)(++dashType)));
    dataCL.ForEach(p => p.dataGroups[i].Format = PF.CloneAs());
    dataEvap.ForEach(p => p.dataGroups[i].Format = PF.CloneAs());
    
    double alpha = 180/Math.PI * 0.5*Math.Asin(2*Convert.ToDouble(sessions[i].KeysAndQueries["id:HeatFlux"]));
    Vals[i] = new Plot2Ddata.XYvalues("q:" + Convert.ToString(sessions[i].KeysAndQueries["id:HeatFlux"]), new double[] {0.0, 15}, new double[] {alpha, alpha});
    Vals[i].Format = new PlotFormat("b-");

    double dt = Convert.ToDouble(sessions[i].KeysAndQueries["dtFixed"]);
    PF.LineWidth = 2.0;
    PF.LineColor = (LineColors)1;
    PF.DashType = (DashTypes)1;
    evapFluxes.dataGroups[i] = new Plot2Ddata.XYvalues("");
    evapFluxes.dataGroups[i].Values = dataEvap.Pick(3).dataGroups[i].Values.Select(x => x).ToArray();
    evapFluxes.dataGroups[i].Abscissas = dataEvap.Pick(3).dataGroups[i].Abscissas.Select(x => x).ToArray();
    evapFluxes.dataGroups[i].Name = "Q-Evap:" + Convert.ToString(sessions[i].KeysAndQueries["id:HeatFlux"]);
    evapFluxes.dataGroups[i].Format = PF.CloneAs();
    evapFluxes.dataGroups[i].Values = evapFluxes.dataGroups[i].Values.Select(v => v / dt).ToArray();

    PF.LineColor = (LineColors)3;
    PF.DashType = (DashTypes)3;
    bndFluxes.dataGroups[i] = new Plot2Ddata.XYvalues("");
    bndFluxes.dataGroups[i].Values = dataCL.Pick(1).dataGroups[i].Values.Select(x => x).ToArray();
    bndFluxes.dataGroups[i].Abscissas = dataCL.Pick(1).dataGroups[i].Abscissas.Select(x => x).ToArray();
    bndFluxes.dataGroups[i].Name = "Q-Bnd:" + Convert.ToString(sessions[i].KeysAndQueries["id:HeatFlux"]);
    bndFluxes.dataGroups[i].Format = PF.CloneAs();
    bndFluxes.dataGroups[i].Values = bndFluxes.dataGroups[i].Values.Select(v => v * Convert.ToDouble(sessions[i].KeysAndQueries["id:HeatFlux"])).ToArray();

    PF.LineColor = (LineColors)color;
    PF.DashType = (DashTypes)dashType;
    clVeloY.dataGroups[i] = new Plot2Ddata.XYvalues("");
    clVeloY.dataGroups[i].Values = new double[dataCL.Pick(3).dataGroups[i].Values.Length];
    for(int j = 0; j<dataCL.Pick(3).dataGroups[i].Values.Length; j++){
        clVeloY.dataGroups[i].Values[j] = dataCL.Pick(3).dataGroups[i].Values[j] - dataCL.Pick(2).dataGroups[i].Values[j] / Math.Tan(Math.PI / 180.0 * dataCL.Pick(4).dataGroups[i].Values[j]);
    } 
    clVeloY.dataGroups[i].Abscissas = dataCL.Pick(1).dataGroups[i].Abscissas.Select(x => x).ToArray();
    clVeloY.dataGroups[i].Name = "Q:" + Convert.ToString(sessions[i].KeysAndQueries["id:HeatFlux"]);
    clVeloY.dataGroups[i].Format = PF.CloneAs();
}
dataCL.Pick(4).dataGroups = dataCL.Pick(4).dataGroups.ToList().Cat(Vals).ToArray();    
evapFluxes = evapFluxes.Merge(bndFluxes);

number of contact lines: 2
Element at 0: time vs contact-pointX
Element at 1: time vs contact-pointY
Element at 2: time vs contact-VelocityX
Element at 3: time vs contact-VelocityY
Element at 4: time vs contact-angle
Element at 0: time vs mass-liq
Element at 1: time vs mass-vap
Element at 2: time vs mass-total
Element at 3: time vs masschange-evap
Element at 4: time vs masschange-vapor
Element at 5: time vs masschange-liquid
Element at 6: time vs masschange-total
Element at 7: time vs interface length


In [ ]:
var gp = new Gnuplot();
var data = dataCL.Pick(4);
data.Xlabel = "time [s]";
data.Ylabel = "angle [°]";
data.ToGnuplot(gp);
gp.PlotNow()

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 20 
 
 
 
 
 40 
 
 
 
 
 60 
 
 
 
 
 80 
 
 
 
 
 100 
 
 
 
 
 120 
 
 
 
 
 140 
 
 
 
 
 0 
 
 
 
 
 2 
 
 
 
 
 4 
 
 
 
 
 6 
 
 
 
 
 8 
 
 
 
 
 10 
 
 
 
 
 12 
 
 
 
 
 14 
 
 
 
 
 16 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 angle [°] 
 
 
 
 
 time [s] 
 
 
 
 
 q:0 
 
 
 q:0 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M610.7,57.1 L664.1,57.1 M71.9,216.9 L72.3,216.9 L72.7,235.0 L73.2,247.2 L73.6,256.9 L74.0,265.1
 L74.4,272.5 L74.9,278.8 L75.3,284.7 L75.7,290.0 L76.1,294.7 L76.6,298.1 L77.0,300.9 L77.4,305.7
 L77.8,309.8 L78.3,313.4 L78.7,316.8 L79.1,320.1 L79.5,323.2 L80.0,326.0 L80.4,325.0 L80.8,328.8
 L81.2,332.1 L81.6,335.0 L82.1,337.7 L82.5,340.4 L82.9,343.0 L83.3,344.7 L83.8,340.7 L84.2,344.2
 L84.6,344.6 L85.0,348.3 L85.5,351.1 L85.9,353.6 L86.3,355.8 L86.7,358.0 L87.2,360.2 L87.6,362.3
 L88.0,364.2 L88.4,366.2 L88.9,368.0 L89.3,367.0 L89.7,369.8 L90.1,372.8 L90.6,369.0 L91.0,371.6
 L91.4,373.0 L91.8,374.2 L92.2,375.5 L92.7,376.7 L93.1,378.1 L93.5,379.4 L93.9,380.7 L94.4,382.0
 L94.8,381.3 L95.2,382.9 L95.6,385.1 L96.1,387.0 L96.5,388.8 L96.9,393.0 L97.3,387.6 L97.8,387.4
 L98.2,389.5 L98.6,390.6 L99.0,391.7 L99.5,392.8 L99.9,394.1 L100.3,395.3 L100.7,396.5 L101.1,397.7
 L101.6,398.8 L102.0,397.5 L102.4,398.7 L102.8,399.9 L103.3,401.1 L103.7,404.8 L104.1,401.4 L104.5,400.8
 L105.0,400.7 L105.4,401.0 L105.8,401.5 L106.2,402.1 L106.7,402.9 L107.1,405.1 L107.5,405.1 L107.9,407.0
 L108.4,408.7 L108.8,408.3 L109.2,409.6 L109.6,410.2 L110.0,411.2 L110.5,412.3 L110.9,411.0 L111.3,410.3
 L111.7,412.1 L112.2,410.4 L112.6,410.9 L113.0,413.0 L113.4,412.4 L113.9,413.3 L114.3,414.5 L114.7,415.4
 L115.1,417.2 L115.6,417.5 L116.0,418.7 L116.4,419.6 L116.8,420.6 L117.3,421.5 L117.7,420.4 L118.1,419.2
 L118.5,418.6 L119.0,418.4 L119.4,418.5 L119.8,418.8 L120.2,419.4 L120.6,421.0 L121.1,420.8 L121.5,421.7
 L121.9,423.1 L122.3,424.2 L122.8,424.0 L123.2,425.0 L123.6,425.8 L124.0,426.5 L124.5,425.2 L124.9,426.2
 L125.3,423.1 L125.7,424.5 L126.2,425.0 L126.6,423.3 L127.0,423.9 L127.4,424.6 L127.9,425.3 L128.3,426.2
 L128.7,427.7 L129.1,429.0 L129.5,430.0 L130.0,429.2 L130.4,430.2 L130.8,431.1 L131.2,430.5 L131.7,428.7
 L132.1,429.7 L132.5,429.6 L132.9,429.6 L133.4,429.8 L133.8,430.2 L134.2,430.8 L134.6,431.4 L135.1,432.0
 L135.5,432.5 L135.9,430.9 L136.3,431.9 L136.8,432.7 L137.2,432.6 L137.6,433.1 L138.0,433.7 L138.4,431.8
 L138.9,432.7 L139.3,432.6 L139.7,430.0 L140.1,431.2 L140.6,432.2 L141.0,431.3 L141.4,432.0 L141.8,433.3
 L142.3,433.6 L142.7,434.7 L143.1,435.8 L143.5,436.7 L144.0,437.4 L144.4,438.1 L144.8,442.8 L145.2,436.0
 L145.7,436.6 L146.1,436.2 L146.5,435.9 L146.9,435.8 L147.3,435.9 L147.8,436.3 L148.2,434.8 L148.6,435.7
 L149.0,436.6 L149.5,436.6 L149.9,437.0 L150.3,437.4 L150.7,437.9 L151.2,438.3 L151.6,443.7 L152.0,437.0
 L152.4,435.5 L152.9,436.1 L153.3,434.3 L153.7,434.3 L154.1,435.1 L154.6,435.8 L155.0,436.4 L155.4,437.1
 L155.8,437.7 L156.3,437.0 L156.7,437.4 L157.1,437.9 L157.5,438.1 L157.9,438.6 L158.4,439.9 L158.8,438.2
 L159.2,438.6 L159.6,436.0 L160.1,435.6 L160.5,436.3 L160.9,436.7 L161.3,437.1 L161.8,436.7 L162.2,437.5
 L162.6,438.4 L163.0,438.9 L163.5,438.6 L163.9,438.8 L164.3,438.9 L164.7,439.6 L165.2,443.2 L165.6,438.9
 L166.0,439.2 L166.4,438.5 L166.8,436.2 L167.3,436.2 L167.7,436.3 L168.1,436.7 L168.5,437.4 L169.0,438.2
 L169.4,438.3 L169.8,438.9 L170.2,439.4 L170.7,439.8 L171.1,440.0 L171.5,440.5 L171.9,445.9 L172.4,440.4
 L172.8,438.9 L173.2,438.0 L173.6,437.5 L174.1,437.4 L174.5,437.8 L174.9,437.9 L175.3,438.4 L175.7,439.0
 L176.2,439.7 L176.6,440.1 L177.0,440.4 L177.4,440.

In [ ]:
var gp = new Gnuplot();
var data = evapFluxes;
data.Xlabel = "time [s]";
data.Ylabel = "Total Heat Flux [W]";
data.ToGnuplot(gp);
gp.PlotNow()

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.1 
 
 
 
 
 0 
 
 
 
 
 0.1 
 
 
 
 
 0.2 
 
 
 
 
 0.3 
 
 
 
 
 0.4 
 
 
 
 
 0.5 
 
 
 
 
 0.6 
 
 
 
 
 0.7 
 
 
 
 
 0 
 
 
 
 
 2 
 
 
 
 
 4 
 
 
 
 
 6 
 
 
 
 
 8 
 
 
 
 
 10 
 
 
 
 
 12 
 
 
 
 
 14 
 
 
 
 
 16 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Total Heat Flux [W] 
 
 
 
 
 time [s] 
 
 
 
 
 Q-Evap:0 
 
 
 
 
 Q-Evap:0 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M555.2,57.1 L608.6,57.1 M71.9,479.1 L72.3,479.1 L72.7,479.1 L73.2,479.1 L73.6,479.1 L74.0,479.1
 L74.4,479.1 L74.9,479.1 L75.3,479.1 L75.7,479.1 L76.1,479.1 L76.6,479.1 L77.0,479.1 L77.4,479.1
 L77.8,479.1 L78.3,479.1 L78.7,479.1 L79.1,479.1 L79.5,479.1 L80.0,479.1 L80.4,479.1 L80.8,479.1
 L81.2,479.1 L81.6,479.1 L82.1,479.1 L82.5,479.1 L82.9,479.1 L83.3,479.1 L83.8,479.1 L84.2,479.1
 L84.6,479.1 L85.0,479.1 L85.5,479.1 L85.9,479.1 L86.3,479.1 L86.7,479.1 L87.2,479.1 L87.6,479.1
 L88.0,479.1 L88.4,479.1 L88.9,479.1 L89.3,479.1 L89.7,479.1 L90.1,479.1 L90.6,479.1 L91.0,479.1
 L91.4,479.1 L91.8,479.1 L92.2,479.1 L92.7,479.1 L93.1,479.1 L93.5,479.1 L93.9,479.1 L94.4,479.1
 L94.8,479.1 L95.2,479.1 L95.6,479.1 L96.1,479.1 L96.5,479.1 L96.9,479.1 L97.3,479.1 L97.8,479.1
 L98.2,479.1 L98.6,479.1 L99.0,479.1 L99.5,479.1 L99.9,479.1 L100.3,479.1 L100.7,479.1 L101.1,479.1
 L101.6,479.1 L102.0,479.1 L102.4,479.1 L102.8,479.1 L103.3,479.1 L103.7,479.1 L104.1,479.1 L104.5,479.1
 L105.0,479.1 L105.4,479.1 L105.8,479.1 L106.2,479.1 L106.7,479.1 L107.1,479.1 L107.5,479.1 L107.9,479.1
 L108.4,479.1 L108.8,479.1 L109.2,479.1 L109.6,479.1 L110.0,479.1 L110.5,479.1 L110.9,479.1 L111.3,479.1
 L111.7,479.1 L112.2,479.1 L112.6,479.1 L113.0,479.1 L113.4,479.1 L113.9,479.1 L114.3,479.1 L114.7,479.1
 L115.1,479.1 L115.6,479.1 L116.0,479.1 L116.4,479.1 L116.8,479.1 L117.3,479.1 L117.7,479.1 L118.1,479.1
 L118.5,479.1 L119.0,479.1 L119.4,479.1 L119.8,479.1 L120.2,479.1 L120.6,479.1 L121.1,479.1 L121.5,479.1
 L121.9,479.1 L122.3,479.1 L122.8,479.1 L123.2,479.1 L123.6,479.1 L124.0,479.1 L124.5,479.1 L124.9,479.1
 L125.3,479.1 L125.7,479.1 L126.2,479.1 L126.6,479.1 L127.0,479.1 L127.4,479.1 L127.9,479.1 L128.3,479.1
 L128.7,479.1 L129.1,479.1 L129.5,479.1 L130.0,479.1 L130.4,479.1 L130.8,479.1 L131.2,479.1 L131.7,479.1
 L132.1,479.1 L132.5,479.1 L132.9,479.1 L133.4,479.1 L133.8,479.1 L134.2,479.1 L134.6,479.1 L135.1,479.1
 L135.5,479.1 L135.9,479.1 L136.3,479.1 L136.8,479.1 L137.2,479.1 L137.6,479.1 L138.0,479.1 L138.4,479.1
 L138.9,479.1 L139.3,479.1 L139.7,479.1 L140.1,479.1 L140.6,479.1 L141.0,479.1 L141.4,479.1 L141.8,479.1
 L142.3,479.1 L142.7,479.1 L143.1,479.1 L143.5,479.1 L144.0,479.1 L144.4,479.1 L144.8,479.1 L145.2,479.1
 L145.7,479.1 L146.1,479.1 L146.5,479.1 L146.9,479.1 L147.3,479.1 L147.8,479.1 L148.2,479.1 L148.6,479.1
 L149.0,479.1 L149.5,479.1 L149.9,479.1 L150.3,479.1 L150.7,479.1 L151.2,479.1 L151.6,479.1 L152.0,479.1
 L152.4,479.1 L152.9,479.1 L153.3,479.1 L153.7,479.1 L154.1,479.1 L154.6,479.1 L155.0,479.1 L155.4,479.1
 L155.8,479.1 L156.3,479.1 L156.7,479.1 L157.1,479.1 L157.5,479.1 L157.9,479.1 L158.4,479.1 L158.8,479.1
 L159.2,479.1 L159.6,479.1 L160.1,479.1 L160.5,479.1 L160.9,479.1 L161.3,479.1 L161.8,479.1 L162.2,479.1
 L162.6,479.1 L163.0,479.1 L163.5,479.1 L163.9,479.1 L164.3,479.1 L164.7,479.1 L165.2,479.1 L165.6,479.1
 L166.0,479.1 L166.4,479.1 L166.8,479.1 L167.3,479.1 L167.7,479.1 L168.1,479.1 L168.5,479.1 L169.0,479.1
 L169.4,479.1 L169.8,479.1 L170.2,479.1 L170.7,479.1 L171.1,479.1 L171.5,479.1 L171.9,479.1 L172.4,479.1
 L172.8,479.1 L173.2,479.1 L173.6,479.1 L174.1,479.1 L174.5,479.1 L174.9,479.1 L175.3,479.1 L175.7,479

In [ ]:
var gp = new Gnuplot();
var data = dataCL.Pick(1);
data.Xlabel = "time [s]";
data.Ylabel = "Pos Y [m]";
data.ToGnuplot(gp);
gp.PlotNow()

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 2 
 
 
 
 
 4 
 
 
 
 
 6 
 
 
 
 
 8 
 
 
 
 
 10 
 
 
 
 
 12 
 
 
 
 
 14 
 
 
 
 
 16 
 
 
 
 
 0 
 
 
 
 
 2 
 
 
 
 
 4 
 
 
 
 
 6 
 
 
 
 
 8 
 
 
 
 
 10 
 
 
 
 
 12 
 
 
 
 
 14 
 
 
 
 
 16 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Pos Y [m] 
 
 
 
 
 time [s] 
 
 
 
 
 q:0 
 
 
 q:0 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M610.7,57.1 L664.1,57.1 M63.6,510.8 L64.0,510.8 L64.5,510.5 L64.9,510.2 L65.3,509.9 L65.7,509.5
 L66.2,509.2 L66.6,508.9 L67.0,508.6 L67.5,508.3 L67.9,508.0 L68.3,507.7 L68.7,507.4 L69.2,507.1
 L69.6,506.8 L70.0,506.5 L70.5,506.2 L70.9,505.9 L71.3,505.6 L71.8,505.2 L72.2,505.0 L72.6,504.6
 L73.0,504.3 L73.5,504.0 L73.9,503.7 L74.3,503.4 L74.8,503.1 L75.2,502.8 L75.6,502.5 L76.0,502.2
 L76.5,501.9 L76.9,501.6 L77.3,501.3 L77.8,500.9 L78.2,500.6 L78.6,500.3 L79.0,500.0 L79.5,499.7
 L79.9,499.4 L80.3,499.1 L80.8,498.8 L81.2,498.5 L81.6,498.2 L82.0,497.9 L82.5,497.5 L82.9,497.2
 L83.3,496.9 L83.8,496.6 L84.2,496.3 L84.6,496.0 L85.1,495.7 L85.5,495.4 L85.9,495.1 L86.3,494.8
 L86.8,494.5 L87.2,494.2 L87.6,493.9 L88.1,493.6 L88.5,493.2 L88.9,492.9 L89.3,492.6 L89.8,492.3
 L90.2,492.0 L90.6,491.7 L91.1,491.4 L91.5,491.1 L91.9,490.8 L92.3,490.5 L92.8,490.2 L93.2,489.8
 L93.6,489.5 L94.1,489.2 L94.5,488.9 L94.9,488.6 L95.4,488.3 L95.8,488.0 L96.2,487.7 L96.6,487.4
 L97.1,487.1 L97.5,486.8 L97.9,486.5 L98.4,486.2 L98.8,485.9 L99.2,485.5 L99.6,485.2 L100.1,484.9
 L100.5,484.6 L100.9,484.3 L101.4,484.0 L101.8,483.7 L102.2,483.4 L102.6,483.1 L103.1,482.8 L103.5,482.4
 L103.9,482.1 L104.4,481.8 L104.8,481.5 L105.2,481.2 L105.6,480.9 L106.1,480.6 L106.5,480.3 L106.9,480.0
 L107.4,479.7 L107.8,479.4 L108.2,479.1 L108.7,478.8 L109.1,478.4 L109.5,478.1 L109.9,477.8 L110.4,477.5
 L110.8,477.2 L111.2,476.9 L111.7,476.6 L112.1,476.3 L112.5,476.0 L112.9,475.7 L113.4,475.4 L113.8,475.1
 L114.2,474.8 L114.7,474.4 L115.1,474.1 L115.5,473.8 L115.9,473.5 L116.4,473.2 L116.8,472.9 L117.2,472.6
 L117.7,472.3 L118.1,472.0 L118.5,471.6 L118.9,471.4 L119.4,471.1 L119.8,470.8 L120.2,470.5 L120.7,470.1
 L121.1,469.8 L121.5,469.5 L122.0,469.2 L122.4,468.9 L122.8,468.6 L123.2,468.3 L123.7,468.0 L124.1,467.7
 L124.5,467.3 L125.0,467.0 L125.4,466.7 L125.8,466.4 L126.2,466.1 L126.7,465.8 L127.1,465.5 L127.5,465.2
 L128.0,464.9 L128.4,464.6 L128.8,464.3 L129.2,464.0 L129.7,463.7 L130.1,463.4 L130.5,463.0 L131.0,462.7
 L131.4,462.4 L131.8,462.1 L132.3,461.8 L132.7,461.5 L133.1,461.2 L133.5,460.9 L134.0,460.6 L134.4,460.3
 L134.8,460.0 L135.3,459.7 L135.7,459.3 L136.1,459.0 L136.5,458.7 L137.0,458.4 L137.4,458.1 L137.8,457.8
 L138.3,457.5 L138.7,457.1 L139.1,456.8 L139.5,456.5 L140.0,456.2 L140.4,455.9 L140.8,455.7 L141.3,455.3
 L141.7,455.0 L142.1,454.7 L142.5,454.4 L143.0,454.1 L143.4,453.8 L143.8,453.5 L144.3,453.2 L144.7,452.9
 L145.1,452.6 L145.6,452.2 L146.0,451.9 L146.4,451.6 L146.8,451.3 L147.3,451.0 L147.7,450.7 L148.1,450.4
 L148.6,450.1 L149.0,449.8 L149.4,449.5 L149.8,449.2 L150.3,448.9 L150.7,448.6 L151.1,448.2 L151.6,447.9
 L152.0,447.6 L152.4,447.3 L152.8,447.0 L153.3,446.7 L153.7,446.4 L154.1,446.1 L154.6,445.8 L155.0,445.5
 L155.4,445.2 L155.8,444.9 L156.3,444.6 L156.7,444.3 L157.1,443.9 L157.6,443.6 L158.0,443.3 L158.4,443.0
 L158.9,442.7 L159.3,442.4 L159.7,442.1 L160.1,441.8 L160.6,441.5 L161.0,441.2 L161.4,440.9 L161.9,440.5
 L162.3,440.3 L162.7,439.9 L163.1,439.6 L163.6,439.3 L164.0,439.0 L164.4,438.7 L164.9,438.4 L165.3,438.1
 L165.7,437.8 L166.1,437.5 L166.6,437.2 L167.0,436.9 L167.4,436.5 L167.9,436.2 L168.3,435.9 L168.7,435.6
 L169.1,435.3 L169.6,435.0 L170.0,434.7 L170.4,434.4 

In [ ]:
var gp = new Gnuplot();
var data = clVeloY;
data.Xlabel = "time [s]";
data.Ylabel = "V_y [m/s]";
data.ToGnuplot(gp);
gp.PlotNow()

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.4 
 
 
 
 
 -0.2 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 0 
 
 
 
 
 2 
 
 
 
 
 4 
 
 
 
 
 6 
 
 
 
 
 8 
 
 
 
 
 10 
 
 
 
 
 12 
 
 
 
 
 14 
 
 
 
 
 16 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 V y [m/s] 
 
 
 
 
 time [s] 
 
 
 
 
 Q:0 
 
 
 
 
 Q:0 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M610.7,57.1 L664.1,57.1 M71.9,397.7 L72.3,397.7 L72.7,66.9 L73.2,57.8 L73.6,55.8 L74.0,54.7
 L74.4,53.8 L74.9,53.0 L75.3,52.2 L75.7,51.4 L76.1,50.4 L76.6,49.3 L77.0,49.0 L77.4,49.8
 L77.8,50.9 L78.3,50.2 L78.7,49.6 L79.1,48.5 L79.5,48.2 L80.0,48.0 L80.4,47.8 L80.8,47.7
 L81.2,47.6 L81.6,47.4 L82.1,47.1 L82.5,46.9 L82.9,46.5 L83.3,46.3 L83.8,46.4 L84.2,46.1
 L84.6,45.8 L85.0,45.5 L85.5,45.3 L85.9,45.0 L86.3,44.4 L86.7,43.8 L87.2,43.6 L87.6,43.5
 L88.0,43.5 L88.4,43.6 L88.9,43.9 L89.3,44.3 L89.7,44.7 L90.1,44.3 L90.6,44.2 L91.0,45.3
 L91.4,44.9 L91.8,44.6 L92.2,44.4 L92.7,44.4 L93.1,44.3 L93.5,44.2 L93.9,44.6 L94.4,44.5
 L94.8,44.7 L95.2,45.0 L95.6,45.5 L96.1,46.1 L96.5,46.8 L96.9,222.2 L97.3,47.1 L97.8,46.2
 L98.2,45.5 L98.6,44.9 L99.0,44.5 L99.5,44.2 L99.9,44.0 L100.3,43.8 L100.7,44.8 L101.1,44.8
 L101.6,45.1 L102.0,45.4 L102.4,46.0 L102.8,46.4 L103.3,46.8 L103.7,221.7 L104.1,46.4 L104.5,46.7
 L105.0,46.1 L105.4,45.7 L105.8,45.4 L106.2,45.1 L106.7,44.8 L107.1,44.5 L107.5,44.2 L107.9,45.5
 L108.4,45.7 L108.8,46.0 L109.2,46.5 L109.6,47.0 L110.0,47.4 L110.5,47.7 L110.9,45.8 L111.3,45.6
 L111.7,45.1 L112.2,44.7 L112.6,44.5 L113.0,44.3 L113.4,44.1 L113.9,44.0 L114.3,43.8 L114.7,44.7
 L115.1,44.8 L115.6,45.1 L116.0,45.5 L116.4,45.9 L116.8,46.3 L117.3,46.5 L117.7,44.4 L118.1,46.9
 L118.5,46.6 L119.0,46.4 L119.4,46.2 L119.8,46.0 L120.2,45.8 L120.6,45.6 L121.1,45.3 L121.5,45.2
 L121.9,47.2 L122.3,47.6 L122.8,47.9 L123.2,48.4 L123.6,48.6 L124.0,48.6 L124.5,46.6 L124.9,46.6
 L125.3,46.1 L125.7,46.0 L126.2,45.9 L126.6,45.7 L127.0,45.6 L127.4,45.4 L127.9,45.2 L128.3,45.0
 L128.7,46.5 L129.1,46.8 L129.5,46.9 L130.0,47.0 L130.4,47.1 L130.8,46.9 L131.2,220.8 L131.7,45.8
 L132.1,46.9 L132.5,46.6 L132.9,46.3 L133.4,46.0 L133.8,45.8 L134.2,45.6 L134.6,45.3 L135.1,45.1
 L135.5,47.2 L135.9,47.4 L136.3,47.8 L136.8,48.1 L137.2,48.3 L137.6,48.4 L138.0,221.1 L138.4,45.7
 L138.9,45.2 L139.3,44.9 L139.7,44.6 L140.1,44.6 L140.6,44.5 L141.0,44.2 L141.4,44.1 L141.8,43.9
 L142.3,43.8 L142.7,45.4 L143.1,45.6 L143.5,45.7 L144.0,45.7 L144.4,45.6 L144.8,219.5 L145.2,43.1
 L145.7,45.7 L146.1,45.7 L146.5,45.7 L146.9,45.7 L147.3,45.6 L147.8,45.5 L148.2,45.3 L148.6,45.6
 L149.0,45.5 L149.5,48.3 L149.9,48.9 L150.3,49.3 L150.7,49.4 L151.2,49.3 L151.6,221.8 L152.0,46.9
 L152.4,46.6 L152.9,46.5 L153.3,46.4 L153.7,46.4 L154.1,46.3 L154.6,46.1 L155.0,45.9 L155.4,45.7
 L155.8,45.5 L156.3,47.5 L156.7,47.9 L157.1,48.0 L157.5,48.0 L157.9,47.7 L158.4,221.1 L158.8,45.6
 L159.2,46.6 L159.6,46.3 L160.1,46.2 L160.5,46.1 L160.9,45.9 L161.3,45.7 L161.8,45.5 L162.2,45.4
 L162.6,45.3 L163.0,48.1 L163.5,48.3 L163.9,48.5 L164.3,48.5 L164.7,48.4 L165.2,221.4 L165.6,46.5
 L166.0,46.0 L166.4,45.7 L166.8,45.5 L167.3,45.6 L167.7,45.5 L168.1,45.3 L168.5,45.2 L169.0,45.0
 L169.4,44.8 L169.8,46.7 L170.2,46.9 L170.7,47.0 L171.1,46.9 L171.5,46.7 L171.9,220.2 L172.4,44.5
 L172.8,46.4 L173.2,46.5 L173.6,46.5 L174.1,46.4 L174.5,46.4 L174.9,46.2 L175.3,46.2 L175.7,46.1
 L176.2,46.0 L176.6,48.8 L177.0,49.0 L177.4,49.1 L177.9,48.9 L178.3,48.7 L178.7,221.5 L179.1,46.6
 L179.6,46.1 L180.0,46.0 L180.4,45.9 L180.8,45.9 L181.3,45.7 L181.7,46.0 L182.1,45.9 L182.5,45.8
 L183.0,45.6 L183.4,47.9 L183.8,48.2 L184.2,48.3 L184.7,48.3 L185.1,48.0 L185.5,221